Imports

In [55]:
from filter_datasets import *
import json
import collections
import ppx #A Python interface to proteomics data repositories
from pyteomics import mztab
from ete3 import Tree, NodeStyle, TreeStyle, NodeStyle, faces, AttrFace, CircleFace
from ete3 import NCBITaxa

Search databases in pride:

- Pride_plant.json is the result from filterPride function on PRIDE databases


In [56]:
with open('pride_plant.json') as pride_plants_file:
    pride_plant = json.load(pride_plants_file)
print(f"Total of plants projects retrieved from PRIDE database: {len(pride_plant)}")

Total of plants projects retrieved from PRIDE database: 1464


- Pride_plantas_mztab.json is the result from get_mztabs on PRIDE plants projects

In [57]:
with open("pride_plants_mztab.json") as mztabs:
    pride_plants_mztab = json.load(mztabs)
print(f"Total of projects with mztab files from PRIDE: {len(pride_plants_mztab)}")

Total of projects with mztab files from PRIDE: 270


In [71]:
pride_plants_names = [values["name"] for keys,values in pride_plant.items()]
pride_plants_counter = collections.Counter(pride_plants_names)
plant_names = [plants[0] for plants in pride_plants_counter.most_common()]
plant_quant = [plants[1] for plants in pride_plants_counter.most_common()]

In [76]:
import seaborn as sn

Matplotlib is building the font cache; this may take a moment.


In [73]:
plant_quant

[490,
 73,
 58,
 57,
 50,
 46,
 46,
 35,
 25,
 21,
 20,
 19,
 17,
 15,
 14,
 14,
 13,
 10,
 8,
 7,
 7,
 7,
 7,
 6,
 6,
 6,
 6,
 6,
 6,
 5,
 5,
 5,
 5,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 